In [ ]:
import os
import logging
import sys
import nest_asyncio
import json
import torch

import numpy as np
from typing import List, Any
from llama_index.core.embeddings import BaseEmbedding

from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core import (
    Settings,
    Document, get_response_synthesizer,
    KnowledgeGraphIndex, PropertyGraphIndex,
    StorageContext
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.graph_stores.neo4j import Neo4jGraphStore

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True, model_kwargs={"torch_dtype":"bfloat16"})

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = model.to(device)
# device

In [ ]:
data = []
with open('../core/data/annotations.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        for doc in line['documents']:
            data.append(doc["text"])
            
data = data[:1000]

In [ ]:
embeddings = model.encode(data, batch_size=32, show_progress_bar=True)
type(embeddings)

In [ ]:
embeddings = embeddings.tolist()
len(embeddings)

In [ ]:
def get_embeddings(text: List[str],
                   model: str,
                   **kwargs) -> List:
    """
    Get embeddings for a given text using a local pre-trained model.
    """
    _model = SentenceTransformer(model, trust_remote_code=True, model_kwargs={"torch_dtype":"bfloat16"})
    embeddings = _model.encode(text, batch_size=32, show_progress_bar=True)
    embeddings = embeddings.tolist()
    return embeddings
    

In [ ]:
embeddings = get_embeddings(data, "Alibaba-NLP/gte-large-en-v1.5")
len(embeddings)